# Popularity Prediction
## Problem 1.1
As a preliminary step, we calculated the following statistics to get a holistic overview of the twitter dataset.

In [6]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

import os
import tqdm #python for loop progress bar

import datetime, time
import pytz



In [7]:
#these values were found via wc -l in CLI
hashtag_dict = {
    'gohawks':188136,
    'gopatriots':26232,
    'nfl':259024,
    'patriots':489713,
    'sb49':826951,
    'superbowl':1348767
}

In [8]:
def build_matrix(raw_df, index='date', mat_type='full'):
    """
    Iterates through the rows of the created dataframe and performs summing, maxes, and 
    creates time series
    """
    raw_df = raw_df.set_index(index)
    time_series = raw_df.groupby(pd.TimeGrouper(freq='60Min'))
    
    X = np.zeros((len(time_series), 8))
    y = np.zeros((len(time_series), 1))
    for i, (time_interval, g) in enumerate(time_series):
        """
        #get the date for sorting
        date = datetime.fromtimestamp(tweet_data['firstpost_date'])
        df.set_value(i, 'date', date)
        df.set_value(i, 'total_tweets', 1)
        df.set_value(i, 'total_retweets', tweet['metrics']['citations']['total'])
        
        #will sum and take max in post processing
        df.set_value(i, 'sum_followers', tweet['author']['followers'])
        df.set_value(i, 'max_followers', tweet['author']['followers'])
        df.set_value(i, 'total_replies', tweet['metrics']['citations']['replies'])
        df.set_value(i, 'total_ranking', tweet['metrics']['ranking_score'])
        df.set_value(i, 'total_impressions', tweet['metrics']['impressions'])


        """
        if(mat_type is 'full'):
            X[i, 0] = g.total_tweets.sum()
            X[i, 1] = g.total_retweets.sum()
            X[i, 2] = g.sum_followers.sum()
            X[i, 3] = g.max_followers.max()
            X[i, 4] = time_interval.hour     #store the hour of the day -> preserve order
            X[i, 5] = g.total_replies.sum()
            X[i, 6] = g.total_ranking.sum()
            X[i, 7] = g.total_impressions.sum()
        elif(mat_type is 'partial'):
            X[i, 0] = g.total_tweets.sum()
            X[i, 1] = g.total_retweets.sum()
            X[i, 2] = g.sum_followers.sum()
            X[i, 3] = g.max_followers.max()
            X[i, 4] = time_interval.hour     #store the hour of the day -> preserve order

            
        y[i, 0] = g.total_tweets.sum()
        
    return np.nan_to_num(X[:-1]), y[1:]

In [11]:
%%time
def load_data(filename=None):
    with open(filename, 'r') as file:
        df = pd.DataFrame(index=range(num_tweets),
                         columns=['date', 'total_tweets', 'total_retweets', 'sum_followers',
                                 'max_followers', 'total_replies', 'total_ranking',
                                 'total_impressions'])

#total_tweets, total_retweets, sum_followers, 
#max_followers, time_of_day, toatl_replies, total_ranking, total_impressions
        for i, l in tqdm.tqdm(enumerate(file), total=num_tweets):

        #pandas df has first element as index and second as col
            tweet = json.loads(l)

            #get the date for sorting
            date = datetime.datetime.fromtimestamp(tweet['firstpost_date'])
            df.set_value(i, 'date', date)
            df.set_value(i, 'total_tweets', 1)
            df.set_value(i, 'total_retweets', tweet['metrics']['citations']['total'])

            #will sum and take max in post processing
            df.set_value(i, 'sum_followers', tweet['author']['followers'])
            df.set_value(i, 'max_followers', tweet['author']['followers'])
            df.set_value(i, 'total_replies', tweet['metrics']['citations']['replies'])
            df.set_value(i, 'total_ranking', tweet['metrics']['ranking_score'])
            df.set_value(i, 'total_impressions', tweet['metrics']['impressions'])

        return df

   # print('#%s' % hashtag)
   # print('\tAvg # of tweets / hour = %.3f' % (total_tweets / total_hours))
   # print('\tAvg # of followers / user = %.3f' % (total_followers / total_users))
   # print('\tAvg # of retweets / tweet = %.3f' % (total_retweets / total_tweets))

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 8.82 µs


In [ ]:
data = {}
for hashtag, num_tweets in hashtag_dict.items():
    print("---")
    print("Hashtag: ", hashtag, " with ntweets: ", num_tweets)
    print("---")
    data[hashtag] = load_data(os.path.join('tweet_data','tweets_#' + hashtag +'.txt'))

  0%|          | 799/188136 [00:00<00:23, 7987.27it/s]

---
Hashtag:  gohawks  with ntweets:  188136
---


  4%|▎         | 923/26232 [00:00<00:02, 9223.81it/s]

---
Hashtag:  gopatriots  with ntweets:  26232
---


  0%|          | 707/259024 [00:00<00:36, 7068.24it/s]

...

  0%|          | 0/489713 [00:00<?, ?it/s]

---
Hashtag:  patriots  with ntweets:  489713
---


100%|██████████| 489713/489713 [00:49<00:00, 9795.44it/s]


---
Hashtag:  sb49  with ntweets:  826951
---


 56%|█████▌    | 462160/826951 [00:50<00:39, 9239.05it/s]

In [ ]:
frames = [data['gohawks'], 
          data['gopatriots'], 
          data['nfl'], 
          data['patriots'], 
          data['sb49'], 
          data['superbowl']]
frame_hashtags=['gohawks', 'gopatriots', 'nfl', 'patriots', 'sb49', 'superbowl']

all_data = pd.concat(frames)

In [ ]:
#Verify that the dataframes are the right size
print(data['gohawks'].shape)
print(data['gopatriots'].shape)
print(data['nfl'].shape)
print(data['patriots'].shape)
print(data['sb49'].shape)
print(data['superbowl'].shape)

assert(data['gohawks'].shape[0] == hashtag_dict['gohawks'])
assert(data['gopatriots'].shape[0] == hashtag_dict['gopatriots'])
assert(data['nfl'].shape[0] == hashtag_dict['nfl'])
assert(data['patriots'].shape[0] == hashtag_dict['patriots'])
assert(data['sb49'].shape[0] == hashtag_dict['sb49'])
assert(data['superbowl'].shape[0] == hashtag_dict['superbowl'])


print(all_data.shape)
s=0
for hashtag, ntweets in hashtag_dict.items():
   s+=ntweets

print(s)

assert(all_data.shape[0] == s)


### a) Here, we show histograms with 1-hour bins that show the number the tweets in hour over time for two hashtag groups, #SuperBowl and #NFL. 

In [ ]:
####TODO
###TODO
##TODO

#frames_to_plot=[data['superbowl'], data['nfl']]
frames_to_plot=[data['gopatriots']]
for df in frames_to_plot:
#    import pdb; pdb.set_trace() # debugging starts here
    total_tweets = df.shape[0]
    
    min_utc = int(df['date'][df.index[0]].utcnow().strftime("%s"))//3600*3600
    #//3600*3600
    max_utc = int(df['date'][df.index[-1]].utcnow().strftime("%s"))//3600*3600
    print(min_utc, max_utc)
    bins = np.arange(min_utc, max_utc+3600, 3600)
    x = []
    for index, row in df.iterrows():
       # print(row['date'].utcnow().strftime("%s"))
        x.append(int(row['date'].utcnow().strftime("%s")))
    n, b, p = plt.hist(x, bins, facecolor='g')
#     print(b,p)
    plt.xlabel('Hour (UTC)')
    plt.ylabel('# of tweets')
    plt.title('#' + hashtag)
#     plt.axis([40, 160, 0, 0.03])
    plt.grid(True)
    plt.show()

## Problem 1.2
For each hashtag, we first fitted a linear regression model using the following five features to
predict the number of tweets in the next hour, with features extracted from tweet data in
the previous hour.
The features we used are:
* Number of tweets (hashtag of interest)
* Total number of retweets (hashtag of interest)
* Sum of the number of followers of the users posting the hashtag
* Maximum number of followers of the users posting the hashtag
* Time of the day (which could take 24 values that represent hours of the day with respect to a given time zone)

For each model, we present the mean squared error and r2 score. Further, we analyzed the significance of each feature using the t-test and P-value, using a third-party statsmodels.api.

In [133]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime, time
import pytz
from itertools import compress
from sklearn import datasets, linear_model
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error

pst_tz = pytz.timezone('US/Pacific')

for df in frames:
    X, y = build_matrix(df, mat_type='partial')
    lr = linear_model.LinearRegression()
    lr.fit(X, y)
    y_pred = lr.predict(X)
    print("\t Mean squared error = %.3f" % (mean_squared_error(y, y_pred)))
    print(sm.OLS(y, X).fit().summary())

	 Mean squared error = 566824.371
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.498
Model:                            OLS   Adj. R-squared:                  0.495
Method:                 Least Squares   F-statistic:                     191.6
Date:                Fri, 16 Mar 2018   Prob (F-statistic):          7.78e-142
Time:                        13:26:13   Log-Likelihood:                -7818.7
No. Observations:                 972   AIC:                         1.565e+04
Df Residuals:                     967   BIC:                         1.567e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1.2

/Users/Jonny/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:1471: RuntimeWarning: divide by zero encountered in double_scalars
  return np.sqrt(eigvals[0]/eigvals[-1])
/Users/Jonny/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1036: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse
/Users/Jonny/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in greater
  return (self.a < x) & (x < self.b)
/Users/Jonny/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:879: RuntimeWarning: invalid value encountered in less
  return (self.a < x) & (x < self.b)
/Users/Jonny/anaconda3/lib/python3.6/site-packages/scipy/stats/_distn_infrastructure.py:1818: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= self.a)


	 Mean squared error = 27345.569
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.660
Model:                            OLS   Adj. R-squared:                  0.658
Method:                 Least Squares   F-statistic:                     263.3
Date:                Fri, 16 Mar 2018   Prob (F-statistic):          3.58e-156
Time:                        13:26:13   Log-Likelihood:                -4458.0
No. Observations:                 683   AIC:                             8926.
Df Residuals:                     678   BIC:                             8949.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            -0.91

	 Mean squared error = 42576118.849
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.786
Model:                            OLS   Adj. R-squared:                  0.785
Method:                 Least Squares   F-statistic:                     703.7
Date:                Fri, 16 Mar 2018   Prob (F-statistic):          1.04e-317
Time:                        13:26:19   Log-Likelihood:                -9824.9
No. Observations:                 963   AIC:                         1.966e+04
Df Residuals:                     958   BIC:                         1.968e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             1

From observing the significance, or p-value, of each feature, it is evident that the total number of tweets for the current hour is not a statistically significant feature as the p-value is larger than 0.05 (assuming alpha at 0.05). Every other feature, however, shows a low p-value, indicating that it contributes greatly our final prediction, and are therefore are valuable features. In general, a low p-value is evidence for the change in the predictor's value to be directly correlated to the change in the response variable, which is what we desire.

Inutitively, this makes sense as the total tweets for the current hour would not affect the the total tweets for the next hour. Tweets do not become frequent when there is large number of tweets prior, instead, they become frequent when the previous tweets is associated with a large number of retweets and followers, signaling traction and actual social impact (at least virutally). 

## Problem 1.3
Design a regression model using any features from the papers you find or other new features you may find useful for this problem. Fit your model on the data of each hashtag and report fitting accuracy and significance of variables.

The first features we attempted to add were total number of "favorites" per hour, total number of replies per hour, and total number of verified tweeters posting that hour. The motivation for looking at the number of verified tweeters was the hypothesis that a verified public figure would be able to influence the number of tweets having to do with a subject he or she tweeted about. Most verified accounts are celebrities. For example, it's likely that if Tom Brady tweeted "#gopatriots", there would be more retweets and comments sharing this hashtag due to the network effect. It turned out that in the dataset, not many users are verified and the p-value indicated that this feature was not important. The model with these features added did signficantly worse and exhibited an R2-value of 0.4.

In [134]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import datetime, time
import pytz
from itertools import compress
from sklearn import datasets, linear_model
from sklearn.metrics import accuracy_score, r2_score, mean_squared_error

pst_tz = pytz.timezone('US/Pacific')

for df in frames:
    X, y = build_matrix(df, mat_type='full')
    lr = linear_model.LinearRegression()
    lr.fit(X, y)
    y_pred = lr.predict(X)
    print("\t Mean squared error = %.3f" % (mean_squared_error(y, y_pred)))
    print(sm.OLS(y, X).fit().summary())

	 Mean squared error = 556864.412
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.507
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     123.7
Date:                Fri, 16 Mar 2018   Prob (F-statistic):          3.31e-142
Time:                        13:26:21   Log-Likelihood:                -7810.0
No. Observations:                 972   AIC:                         1.564e+04
Df Residuals:                     964   BIC:                         1.567e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             7.6

	 Mean squared error = 18516861.767
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.820
Method:                 Least Squares   F-statistic:                     332.2
Date:                Fri, 16 Mar 2018   Prob (F-statistic):          8.72e-210
Time:                        13:26:25   Log-Likelihood:                -5696.0
No. Observations:                 582   AIC:                         1.141e+04
Df Residuals:                     574   BIC:                         1.144e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            12

While we initially started with the following features:
- total number of favorites per hour
- total number of replies per hour
- total number of verified tweeters that hour

Favorites and verified tweets were ineffective features. There are very few verified accounts so it resulted in sparsity and not much information gain. The number of favorites was also pretty low. The total number of replies seemed to improve performance, so we left it in. The next test involved the following three features, after having removed favorites and verified tweeters:
- total number of replies per hour
- total ranking 
- total impressions

Ranking and impressions indicate the popularity or visibility of a tweet/tweeter, so these seemed like reasonable additions. These features improved RMSE performance by several points in terms of R2, and were vastly more effective than the previously added sparse features.


## Problem 1.4
In this part, we would like to perform 10-fold cross-validation on the models from the previous part and calculate the average prediction error over samples in the held-out part for the 10 tests. For this problem, you should split the feature data (your set of (features, predictant) pairs for windows) into 10 parts to perform cross-validation. Also, your evaluated error should be of the form |Npredicted − Nreal|.



In [135]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error #RMSE
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

def cv(X, y, model, n_splits=10, verbose=True, display_last_ols=False):
    kf = KFold(n_splits=n_splits)
    rmses = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        y_train = y_train.ravel()
        y_test = y_test.ravel()
        #print(sm.OLS(y, X).fit().summary())
#        print(X_train.shape)
#        print(y_train.shape)
        if(model == 'lr'):
            lr = sm.OLS(y_train, X_train).fit()
            y_preds = lr.predict(X_test)
        elif(model == 'rf'):
            rf = RandomForestRegressor()
            rf.fit(X_train, y_train)
            y_preds = rf.predict(X_test)
        elif(model == 'mlp'):
            mlp = MLPRegressor()
            mlp.fit(X_train, y_train)
            y_preds = mlp.predict(X_test)
        
        rmses.append(mean_squared_error(y_test, y_preds))
        
        
    if verbose: 
        print("Errors from CV are: ", rmses)
        print("Averaged error is: ", np.mean(rmses))
        if model is 'lr':
            print(lr.summary())
        
    return rmses
    

In [136]:

#datetime args: Attributes: year, month, day, hour, minute, second, microsecond, and tzinfo.
#before Feb 1, 8:00am
first_date_marker = datetime.datetime(2015, 2, 1, 8, 0, 0, 0)

#end at 8pm
second_date_marker = datetime.datetime(2015, 2, 1, 20, 0, 0, 0)



In [148]:
def filter_and_test(df, model, verbose=True):
    ###Set up the data by filtering via index
    #Before Feb. 1, 8:00 a.m.
    #sort out the times in the dataframe before this period
    df_p1 = df[df.date < first_date_marker]


    #Between Feb. 1, 8:00 a.m. and 8:00 p.m. 
    df_p2 = df[(df.date > first_date_marker) &
               (df.date < second_date_marker)]

    #After Feb. 1, 8:00 p.m.
    df_p3 = df[df.date > second_date_marker]

#    print("Before Feb. 1, 8:00 a.m.")
    X_df_p1, y_df_p1 = build_matrix(df_p1, index='date')
    errors_df_p1 = cv(X_df_p1, y_df_p1, model, verbose=verbose) #default splits = 10 no need to specify
   # print("Average cv error: ", np.mean(errors_df_p1))
    
 #   print("Between Feb. 1, 8:00 a.m. and 8:00 p.m.")
    X_df_p2, y_df_p2 = build_matrix(df_p2, index='date')
    errors_df_p2 = cv(X_df_p2, y_df_p2, model,verbose=verbose)
  #  print("Average cv error: ", np.mean(errors_df_p2))
    
  #  print("After Feb. 1, 8:00 p.m.")
    X_df_p3, y_df_p3 = build_matrix(df_p3, index='date')
    errors_df_p3 = cv(X_df_p3, y_df_p3, model,verbose=verbose)
   # print("Average cv error: ", np.mean(errors_df_p3))

    return np.mean(errors_df_p1), np.mean(errors_df_p2), np.mean(errors_df_p3)


### Per-Hashtag Performance with 3 Models
For each hashtag, report the average cross-validation errors for the 3 different models. Note that you should do the 90-10% splitting for each model within its specific time window. I.e. Only use data within one of the 3 periods above for training and testing each time, so for each period you will run 10 tests.

In [145]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return True;
}

<IPython.core.display.Javascript object>

In [150]:
%%time
import os
import tqdm


models = ['lr', 'rf', 'mlp']
results = {}
for model in models:
    print("Model is: "+model)
    i = 0
    p1_errs, p2_errs, p3_errs = [],[],[]
    for df in frames:
        err_p1, err_p2, err_p3 = filter_and_test(df, model, verbose=False)
#        print("Hashtag: ", frame_hashtags[i])
 #       print("Period 1 avg. cv error: ", err_p1)
  #      print("Period 2 avg. cv error: ", err_p2)
   #     print("Period 3 avg. cv error: ", err_p3)
        p1_errs.append(err_p1)
        p2_errs.append(err_p2)
        p3_errs.append(err_p3)
        i+=1
    results[model] = [np.mean(p1_errs), np.mean(p2_errs), np.mean(p3_errs)]
            
#            full_errors = cv(X_full, y_full, n_splits=10)
 #           print("Errors from full set CV: ", full_errors)
  #          print("Mean error from CV: ", np.mean(full_errors))
